#LESSON 8: SECURE FEDERATED LEARNING FOR DEEPLEARNING PROJECT 


**Author** 
: Ateniola Oluwatobi Victor

**Objective** : My implementation of the Final project in the Secure Federated Learning
section of the Secure and Private AI Scholarship Challenge Nanodegree Program using the MNIST digit dataset

In [ ]:
!pip install syft==0.2.9

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 433 kB 5.2 MB/s 
     |████████████████████████████████| 484 kB 41.9 MB/s 
     |████████████████████████████████| 2.0 MB 24.1 MB/s 
     |████████████████████████████████| 1.8 MB 46.7 MB/s 
     |████████████████████████████████| 20.6 MB 12.6 MB/s 
     |████████████████████████████████| 78 kB 2.8 MB/s 
     |████████████████████████████████| 449 kB 16.6 MB/s 
     |████████████████████████████████| 66 kB 805 kB/s 
     |████████████████████████████████| 9.0 MB 30.7 MB/s 
     |████████████████████████████████| 4.0 MB 31.4 MB/s 
     |████████████████████████████████| 57 kB 2.3 MB/s 
     |████████████████████████████████| 54 kB 1.2 MB/s 
     |████████████████████████████████| 126 kB 50.4 MB/s 
     |████████████████████████████████| 200 kB 50.7 MB/s 
     |████████████████████████████████| 753.4 MB 20 kB/s 
     |████████████████████████████████|

In [ ]:
!pip install tensorflow-privacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 301 kB 5.2 MB/s 
     |████████████████████████████████| 89 kB 6.4 MB/s 
     |████████████████████████████████| 9.4 MB 52.7 MB/s 
     |████████████████████████████████| 33.8 MB 1.5 MB/s 
     |████████████████████████████████| 5.7 MB 42.9 MB/s 
     |████████████████████████████████| 12.2 MB 29.1 MB/s 
     |████████████████████████████████| 60 kB 6.9 MB/s 
     |████████████████████████████████| 17.1 MB 62 kB/s 
     |████████████████████████████████| 126 kB 51.7 MB/s 
     |████████████████████████████████| 39.3 MB 1.3 MB/s 
     |████████████████████████████████| 965 kB 49.7 MB/s 
     |████████████████████████████████| 295 kB 60.3 MB/s 
     |████████████████████████████████| 16.9 MB 163 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
 

In [ ]:
!pip install numpy --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.23.5-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.1 requires torch==1.12.1, but you have torch 1.4.0 which is incompatible.
syft 0.2.9 requires numpy~=1.18.1, but you have numpy 1.23.5 which is incompatible.
syft 0.2.9 requires scipy~=1.4.1, but you have scipy 1.7.3 which is incompatible.
scipy 1.7.3 requires numpy<1.23.0,>=1.16.5, but you have numpy 1.23.5 which is incompatible.
google-colab 1.0.0 requires notebook~=5.7.16, but you have notebook 5.7.8 which is incompatible.
google-colab 1.0.0 requires request

In [ ]:
import torch as th
import numpy as np
from torchvision import datasets, transforms
import torchvision.datasets as datasets
from torch.utils.data import Subset
from torch import nn
import torch.nn.functional as F
from torch import optim
import syft as sy
import helper

#Hooking syft to torch
hook = sy.TorchHook(th)

In [ ]:
#Method to create 10 virtual workers and move to a list of workers
def create_workers():
  workers = []
  cartman = sy.VirtualWorker(hook, id = "cartman")
  workers.append(cartman)
  kyle = sy.VirtualWorker(hook, id = "kyle")
  workers.append(kyle)
  kenny = sy.VirtualWorker(hook, id = "kenny")
  workers.append(kenny)
  stan = sy.VirtualWorker(hook, id = "stan")
  workers.append(stan)
  butters = sy.VirtualWorker(hook, id = "butters")
  workers.append(butters)
  wendy = sy.VirtualWorker(hook, id = "wendy")
  workers.append(wendy)
  heidi = sy.VirtualWorker(hook, id = "heidi")
  workers.append(heidi)
  bebe = sy.VirtualWorker(hook, id = "bebe")
  workers.append(bebe)
  nichole = sy.VirtualWorker(hook, id = "nichole")
  workers.append(nichole)
  patty = sy.VirtualWorker(hook, id = "patty")
  workers.append(patty)
  return workers


In [ ]:
#Method to clear out every tensor stored in the list of virtual workers
def clear_workers(workers):
  for worker in workers:
    worker.clear_objects()

In [ ]:
#Method to split the mnist test dataset into the various workers and also to load the mnist test dataset into a test loader
def create_federated_and_test_loaders(workers, trainset, testset):
  federated_train_loader = sy.FederatedDataLoader(
      trainset.federate(workers), 
      batch_size=32, shuffle=True)

  test_loader = th.utils.data.DataLoader(testset, batch_size=64, shuffle=False)
  return federated_train_loader, test_loader

In [ ]:
#Method to bind remote optimizer functions to the Initial states of the remote models 
def create_models(workers, lr):
  remoteModels = list()
  remoteOptimizers = list()
  for worker in workers:
    model = classifier()
    model = model.send(worker)
    remoteOptimizers.append(optim.SGD(model.parameters(), lr))
    remoteModels.append(model)
  return remoteModels, remoteOptimizers
  

In [ ]:
#Method to train models on the virtual workers without moving any gradients to the central model until the gradients have been collated.
def create_train_federated_models(workers, loader, lr = 0.12, epoch = 5):
  #sends model to first virtual worker
  virtual_models, virtual_optimizers = create_models(workers, lr)
#   virtual_model = classifier().send(workers[0])
#   optimizer = optim.SGD(virtual_model.parameters(), lr)
  criterion = nn.NLLLoss()
  for n in range(epoch):
    
    #Integer to keep up with first index.
    i = 0
    
    #Integer to keep up with current worker while training
    j = 0
    
    #Integer to count number of mini-batches per worker
    n_mbatch = 0
    
    #Variable to keep up with current worker while looping
    dbLoc = None
    
    #Variable to store cummulative loss.
    cum_loss = 0
    
    
    for batch_idx, (imgs, labels) in enumerate(loader):
      
      #condition to set dbLoc to the first worker
      if i == 0:
        i = 2
        dbLoc = imgs.location
        
      #condition to change dbLoc if img is stored on a different worker and also calculate loss
      if dbLoc is not imgs.location:
        print("The total loss for {0} for epoch {2} is {1}".format(workers[j].id, cum_loss / n_mbatch, n+1))
        dbLoc = imgs.location
        j += 1
        
        #Moving the model to a new worker
#         virtual_model.move(dbLoc)
        
        #Resetting the cummulative loss and batch count to zero for new worker
        cum_loss = 0
        n_mbatch = 0

      virtual_optimizers[j].zero_grad()
      output = virtual_models[j].forward(imgs)
      loss = criterion(output, labels)
      loss.backward()
      virtual_optimizers[j].step()
      cum_loss +=  loss.get().item()
      n_mbatch += 1
    print("The total loss for {0} is {1}".format(workers[j].id, cum_loss / n_mbatch))
  return virtual_models
  
    
  

In [ ]:
#Method to return the model to the central machine
def create_central_model(model):
  return model.get()

In [ ]:
#Method to convert the gradients of the remote models to their integar representation, encrypt and distribute among the provided workers
def share_gradients(models, workers):
  for model in models:
    model.fc1.weight.data = model.fc1.weight.data.fix_prec().share(*workers)
    model.fc1.bias.data = model.fc1.bias.data.fix_prec().share(*workers)
    model.fc2.weight.data = model.fc2.weight.data.fix_prec().share(*workers)
    model.fc2.bias.data = model.fc2.bias.data.fix_prec().share(*workers)
    model.fc3.weight.data = model.fc3.weight.data.fix_prec().share(*workers)
    model.fc3.bias.data = model.fc3.bias.data.fix_prec().share(*workers)
    model.fc4.weight.data = model.fc4.weight.data.fix_prec().share(*workers)
    model.fc4.bias.data = model.fc4.bias.data.fix_prec().share(*workers)
    model.fc5.weight.data = model.fc5.weight.data.fix_prec().share(*workers)
    model.fc5.bias.data = model.fc5.bias.data.fix_prec().share(*workers)   
  return models
  

In [ ]:
#Method to aggregate the gradients accross the shared devices and return the parameters
def aggregate_grads(models):
  fc1_weight = list()
  fc1_bias = list()
  fc2_weight = list()
  fc2_bias = list()
  fc3_weight = list()
  fc3_bias = list()
  fc4_weight = list()
  fc4_bias = list()
  fc5_weight = list()
  fc5_bias = list()
  for model in models:
    fc1_weight.append(model.fc1.weight.data.clone().get())
    fc1_bias.append(model.fc1.bias.data.clone().get())
    fc2_weight.append(model.fc2.weight.data.clone().get())
    fc2_bias.append(model.fc2.bias.data.clone().get())
    fc3_weight.append(model.fc3.weight.data.clone().get())
    fc3_bias.append(model.fc3.bias.data.clone().get())
    fc4_weight.append(model.fc4.weight.data.clone().get())
    fc4_bias.append(model.fc4.bias.data.clone().get())
    fc5_weight.append(model.fc5.weight.data.clone().get())
    fc5_bias.append(model.fc5.bias.data.clone().get())
  params = {}
  params["fc1.weight"] = (sum(fc1_weight) / len(fc1_weight)).get().float_prec()
  params["fc1.bias"] = (sum(fc1_bias) / len(fc1_bias)).get().float_prec()
  params["fc2.weight"] = (sum(fc2_weight) / len(fc2_weight)).get().float_prec()
  params["fc2.bias"] = (sum(fc2_bias) / len(fc2_bias)).get().float_prec()
  params["fc3.weight"] = (sum(fc3_weight) / len(fc3_weight)).get().float_prec()
  params["fc3.bias"] = (sum(fc3_bias) / len(fc3_bias)).get().float_prec()
  params["fc4.weight"] = (sum(fc4_weight) / len(fc4_weight)).get().float_prec()
  params["fc4.bias"] = (sum(fc4_bias) / len(fc4_bias)).get().float_prec()
  params["fc5.weight"] = (sum(fc5_weight) / len(fc5_weight)).get().float_prec()
  params["fc5.bias"] = (sum(fc5_bias) / len(fc5_bias)).get().float_prec()
  return params


In [ ]:
#Method to update the central model with the aggregated parameters from the remote models
def update_central_model(params):
  model = classifier()
  model.fc1.weight.data = params["fc1.weight"]
  model.fc1.bias.data = params["fc1.bias"]
  model.fc2.weight.data = params["fc2.weight"]
  model.fc2.bias.data = params["fc2.bias"]
  model.fc3.weight.data = params["fc3.weight"]
  model.fc3.bias.data = params["fc3.bias"]
  model.fc4.weight.data = params["fc4.weight"]
  model.fc4.bias.data = params["fc4.bias"]
  model.fc5.weight.data = params["fc5.weight"]
  model.fc5.bias.data = params["fc5.bias"]  
  return model

In [ ]:
#Classifier for creating the models
class classifier(nn.Module):
  def __init__(self):
    super().__init__() 
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 64)
    self.fc4 = nn.Linear(64, 32)
    self.fc5 = nn.Linear(32, 10)
    
  def forward(self, x):
    x = x.view(x.shape[0], -1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = F.relu(self.fc4(x))
    x = F.log_softmax(self.fc5(x), dim = 1)   
    return x
 

In [ ]:
# Application of transforms to normalize the mnist data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [ ]:
workers = create_workers()
clear_workers(workers)

In [ ]:
federatedset, testset= create_federated_and_test_loaders(workers, mnist_trainset, mnist_testset)

In [ ]:
remoteModels = create_train_federated_models(workers, federatedset, lr = 0.12, epoch = 6)

The total loss for cartman for epoch 1 is 1.8967746454984584
The total loss for kyle for epoch 1 is 1.8794371837631185
The total loss for kenny for epoch 1 is 1.9701746473921107
The total loss for stan for epoch 1 is 1.8023722641011501
The total loss for butters for epoch 1 is 1.9363915061062955
The total loss for wendy for epoch 1 is 1.8917610413850623
The total loss for heidi for epoch 1 is 1.8627142601824822
The total loss for bebe for epoch 1 is 1.8502937447517476
The total loss for nichole for epoch 1 is 1.9047379389088204
The total loss for patty is 1.840519645112626
The total loss for cartman for epoch 2 is 0.8729240069998071
The total loss for kyle for epoch 2 is 0.9500277031292307
The total loss for kenny for epoch 2 is 1.1150368650225884
The total loss for stan for epoch 2 is 0.795447595813807
The total loss for butters for epoch 2 is 0.9765887640892191
The total loss for wendy for epoch 2 is 0.891034630067805
The total loss for heidi for epoch 2 is 0.9519690787221523
The tot

In [ ]:
remoteModels = share_gradients(remoteModels, workers[5:8])


In [ ]:
params  = aggregate_grads(remoteModels)

In [ ]:
central_model = update_central_model(params)
